In [1]:
from telethon.sync import TelegramClient
import json
from datetime import datetime
import time
import math
import pandas

session_name = 'qwerty'
api_id = 28379836
api_hash = '2da54c005fba5e954bc7981549f0b55c'
chat = 'https://t.me/dia_token_tracker'

colnames = ["tokenName", "marketCap", "messageUrl", "messageDate", "messageUnixTime"]
data = pandas.read_csv('data.csv', names=colnames)
tokens = data.tokenName.tolist()

weirdTokens = [
    "https://www.dextools.io/app/en/ether/pair-explorer/0x4f9766bc88eb0D42056d48E2DDC4D1Ff2f06018C",
    "https://www.dextools.io/app/en/ether/pair-explorer/0xfE42d0A182E225A058a901E13cfa3A7Ad64554cD"
]


def getMC(message):
    mc = message.partition("\nMarket Cap: $")[2].partition("\n")[0]
    return int(mc.replace(',', ''))

def getTokenName(message):
    return message.partition("(")[2].partition(")")[0]

def getUrl(jsonData):
    length = len(jsonData["entities"])
    element = 0
    while element < length:
        if "url" in jsonData["entities"][element]:
            return jsonData["entities"][element]["url"]
        element = element + 1

    return -1

dt = datetime.fromtimestamp(1682196059)
async with TelegramClient(session_name, api_id, api_hash) as client:
    async for message in client.iter_messages(chat, reverse=True, offset_date = dt):
        jsonData = json.loads(message.to_json())
        #print(jsonData)

        tokenName = getTokenName(jsonData["message"])
        if tokenName in tokens: continue

        marketCap = getMC(jsonData["message"])
        messageDate = datetime.fromisoformat(jsonData["date"])
        messageUnixTime = math.trunc(time.mktime(messageDate.timetuple()))
        messageUrl = getUrl(jsonData)
        if(messageUrl == -1 or messageUrl in weirdTokens): continue

        #print(f"Date is: {messageDate} --- UnixTime is: {messageUnixTime} --- TokenName is: {tokenName} --- MarketCap is:{marketCap} --- Url is: {messageUrl}")
        #print("|||||||||||||||||||||||||")

        tokens.append(tokenName)

        with open('data.csv', 'a') as fd:
            fd.write(f"{tokenName}, {marketCap}, {messageUrl}, {messageDate}, {messageUnixTime}\n") 
